In [287]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time, os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import re 
import string
from textblob import TextBlob
from nltk.tokenize import sent_tokenize,word_tokenize,MWETokenizer, RegexpTokenizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('punkt')
from nltk.util import ngrams
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer,WordNetLemmatizer
from gensim import corpora, models, similarities, matutils



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nickpondok/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nickpondok/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [261]:
!pip3 install -U spacy
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 2.5 MB/s eta 0:00:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
import spacy

In [310]:
reviews_df = pd.read_csv('shang_chi_reviews.csv')
reviews_df

,Unnamed: 0,review
0,0,shang chi review spectacular visual wonder m...
1,1,the much awaited shangi chi and the legend of ...
2,2,like many i was concerned that this new marve...
3,3,no movie is without flaws but this was a brea...
4,4,hands down one of the best mcu movies every a...
...,...,...
2201,2201,marvel is back 👍😘😍 😎😎😎😎😎😎😎😎😎😎😎😎😎😎
2202,2202,boring
2203,2203,⭐⭐⭐⭐⭐⭐ ...
2204,2204,thats what she said


In [311]:
reviews_df.drop('Unnamed: 0',axis=1,inplace=True)

In [312]:
reviews_df

,review
0,shang chi review spectacular visual wonder m...
1,the much awaited shangi chi and the legend of ...
2,like many i was concerned that this new marve...
3,no movie is without flaws but this was a brea...
4,hands down one of the best mcu movies every a...
...,...
2201,marvel is back 👍😘😍 😎😎😎😎😎😎😎😎😎😎😎😎😎😎
2202,boring
2203,⭐⭐⭐⭐⭐⭐ ...
2204,thats what she said


In [339]:
new_reviews_list=[]
for review in reviews_df['review'].values:
    new_reviews_list.append(review)
new_reviews_list

['shang chi review   spectacular visual wonder my rating       cast   performances    the film has simu liu in top form  he is your ordinary guy but extraordinary powers and actor blends into the character well   with not a single false note  giving him good company are actresses awkwafina who is effortless in whatever she does and meng er zhang who also performs some mind numbing stunts   the star of the film of course is veteran actor tony leung  a father who wants to bring back his family together but also is aware of his extraordinary powers  it s his love for his dead wife that makes him tough and cold towards his children  years back leung had charmed the world with his subtle style in wong kar wai s  in the mood for love   decades later  leung has aged like fine wine and it s a delight to watch him play the antagonist with such style   ben kingsley and michelle yeoh too make their presence felt in the limited screen time they have    technical values    shang chi scores high on 

In [483]:
nlp = spacy.load('en_core_web_sm')




def compound_term(review_list):
    
    compounded_list =[]
    for review in review_list:
        mwe_tokenizer = MWETokenizer([('shang','chi'), ('simu', 'liu'),('tony','leung'),('fala','chen')])
        mwe_tokens = mwe_tokenizer.tokenize(word_tokenize(review))
        compounded_list.append(' '.join(mwe_tokens))
    return compounded_list
#################################

def spell_check(review_list):
    new_list =[]
    for review in review_list:
        blob = TextBlob(review)
        new_list.append(blob.correct())
    return (blob_to_list(new_list))

################################

def blob_to_list(blob_list):
    full_blob_list=[]
    for blob in blob_list:
        updated_blob=[]
        blob=blob.split(" ")
        for word in blob:
            updated_blob.append(word)
        final_blob = ' '.join(updated_blob)
        full_blob_list.append(final_blob)

    return full_blob_list
###################
def show_lemmas(list):
    blob_list=[]
    for item in list:
        nlp = spacy.load('en_core_web_sm')
        text=nlp(item)
        words_list=[]
        for token in text:
            lemma_words=token.lemma_
            words_list.append(lemma_words)
            blob=' '.join(words_list)
        blob_list.append(' '.join(words_list))
    return(blob_list)

# def final_blob(blob_list):
#     final_blob_list=[]
#     for blob in blob_list:
#         doc = nlp(blob)
#         final_blob_list.append(show_lemmas(doc))
#     final_blob2=[]
#     for blob2 in final_blob_list:
#         final_blob2.append(' '.join(blob2))
#     return(final_blob2)
########################################

############################
def pre_process(review):
    list_blobs = blob_to_list(review)
    compounded=compound_term(list_blobs)
    spell_checked=spell_check(compounded)
#     list_blobs = blob_to_list(spell_checked)
    lemmatized=show_lemmas(spell_checked)
    return lemmatized
    
    






In [469]:
#compound_term(new_reviews_list)

In [485]:
final_reviews=pre_process(new_reviews_list)




In [486]:
reviews_df = pd.DataFrame(final_reviews,columns=['review'])
reviews_df

,review
0,shang_chi review spectacular visual wonder my ...
1,the much await change chi and the legend of th...
2,like many I be concerned that this new marvel ...
3,no movie be without flaw but this be a breath ...
4,hand down one of the good you move every actor...
...,...
2201,marvel be back 👍 😘 😍 😎 😎 😎 😎 😎 😎 😎 😎 😎 😎 😎 😎 😎 😎
2202,bore
2203,⭐ ⭐ ⭐ ⭐ ⭐ ⭐ amazing
2204,that what she say


In [497]:
reviews_df.to_csv('preprocessed_reviews.csv')
new_reviews = pd.read_csv('preprocessed_reviews.csv')
new_reviews.drop('Unnamed: 0',axis=1,inplace=True)
new_reviews.head()

,review
0,shang_chi review spectacular visual wonder my ...
1,the much await change chi and the legend of th...
2,like many I be concerned that this new marvel ...
3,no movie be without flaw but this be a breath ...
4,hand down one of the good you move every actor...
